In [1]:
import pandas as pd
from dhis2 import Api

from src.main.common import FileUtil
from src.main.common import LoggingConfig
from src.main.application.income import *
from src.main.application.service import *
from src.infrastructure.forms import *
from src.infrastructure.adapters import *

In [2]:
credentials = FileUtil.load_credentias()
username = credentials['username']
password = credentials['password']

url = 'https://dhis-ao.icap.columbia.edu'
api = Api(url, username, password)

In [3]:
# Get the list of supported facilities
org_units = api.get('organisationUnitGroups/gH2DlwAo1ja', params={'fields':'organisationUnits'}).json()['organisationUnits']

In [4]:
# period of analysis
start_period = '2024-04-01'
end_period = '2024-06-30'
period = '2024Q2'

In [5]:
hts_data = pd.DataFrame(columns=['dataElement','period','orgUnit', 'categoryOptionCombo','attributeOptionCombo', 'value'])
hts_data.to_csv('HTS_DATA.csv', index=False)

patient_event_form = PatientEventForm(api)
patient_demographic_form = PatientDemographicForm(api)
event_port = EventAdapter(patient_event_form)
patient_demographics_port = PatientDemographicsAdapter(patient_demographic_form)

compute_hts_vct = ComputeHtsVctService(event_port, patient_demographics_port)
vct_metadata_port = VctIndicatorMetadataAdapter(api)
vct_patients_disaggregation = ComputeHtsDisaggregationService(vct_metadata_port)

compute_hts_pregnant = ComputeHtsPregnantLdService(event_port, patient_demographics_port)
pregnant_metadata_port = PregnantIndicatorMetadataAdapter(api)
pregnant_patients_disaggregation = ComputeHtsDisaggregationService(pregnant_metadata_port)

compute_hts_breastfeeding = ComputeHtsBreastfeedingService(event_port, patient_demographics_port)
breastfeeding_metadata_port = BreastfeedingIndicatorMetadataAdapter(api)
breastfeeding_patients_disaggregation = ComputeHtsDisaggregationService(breastfeeding_metadata_port)

compute_hts_other_pitc = ComputeHtsOtherPitcService(event_port, patient_demographics_port)
other_pitc_metadata_port = OtherPitcIndicadorMetadataAdapter(api)
other_pitc_patients_disaggregation = ComputeHtsDisaggregationService(other_pitc_metadata_port)

compute_hts_emergence_ward = ComputeHtsEmergenceWardService(event_port, patient_demographics_port)
emergence_ward_port = EmergenceWardIndicatorMetadataAdapter(api)
emergenc_ward_patients_disaggregation = ComputeHtsDisaggregationService(emergence_ward_port)

compute_hts_inpatient = ComputeHtsInpatientService(event_port, patient_demographics_port)
inpatient_port = InpatientIndicatorMetadataAdapter(api)
inpatient_patients_disaggregation = ComputeHtsDisaggregationService(inpatient_port)

compute_hts_malnutrition = ComputeHtsMalnutritionService(event_port, patient_demographics_port)
malnutrition_port = MalnutritionIndicatorMetadataAdapter(api)
malnutrition_patients_disaggregation = ComputeHtsDisaggregationService(malnutrition_port)

# compute_hts_index_case = ComputeHtsIndexCaseService(event_port)
# index_case_port = IndexCaseIndicatorMetadataAdapter(api)
# index_case_patients_disaggregation = ComputeHtsDisaggregationService(index_case_port)

for org_unit in org_units:

    org_unit = org_unit['id']

    vct_patients = compute_hts_vct.compute(org_unit, start_period, end_period)
    vct_patients_disaggregated = vct_patients_disaggregation.compute(vct_patients, end_period)

    pregnant_patients = compute_hts_pregnant.compute(org_unit, start_period, end_period)
    pregnant_patients_disaggregated = pregnant_patients_disaggregation.compute(pregnant_patients, end_period)

    breastfeeding_patients = compute_hts_breastfeeding.compute(org_unit, start_period, end_period)
    breastfeeding_patients_disaggregated = breastfeeding_patients_disaggregation.compute(breastfeeding_patients, end_period)

    other_pitc_patients = compute_hts_other_pitc.compute(org_unit, start_period, end_period)
    other_pitc_patients_disaggregated = other_pitc_patients_disaggregation.compute(other_pitc_patients, end_period)

    emergence_ward_patients = compute_hts_emergence_ward.compute(org_unit, start_period, end_period)
    emergence_ward_patients_disaggregated = emergenc_ward_patients_disaggregation.compute(emergence_ward_patients, end_period)

    inpatients = compute_hts_inpatient.compute(org_unit, start_period, end_period)
    inpatients_disaggregated = inpatient_patients_disaggregation.compute(inpatients, end_period)

    maltnutrition_patients = compute_hts_malnutrition.compute(org_unit, start_period, end_period)
    malnutrition_patients_disaggregated = malnutrition_patients_disaggregation.compute(maltnutrition_patients, end_period)

    combination = vct_patients_disaggregated + pregnant_patients_disaggregated + breastfeeding_patients_disaggregated + other_pitc_patients_disaggregated + emergence_ward_patients_disaggregated + inpatients_disaggregated + malnutrition_patients_disaggregated

    if combination:
        hts = pd.json_normalize(combination)
        hts['period'] = period
        hts = hts[['dataElement','period','orgUnit', 'categoryOptionCombo','attributeOptionCombo', 'value']]
        hts_data = pd.read_csv('HTS_DATA.csv')
        hts_data = pd.concat([hts_data, hts])
        hts_data = hts_data.sort_values(['orgUnit', 'dataElement', 'period'])
        hts_data.to_csv('HTS_DATA.csv', index=False)